In [1]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (128, 128))
            images.append(img)
    return images

In [3]:
with_mask_images = load_images_from_folder("C:\Users\Lenovo\Desktop\git\DataSet.zip\DataSet\WithMask")
without_mask_images = load_images_from_folder("C:\Users\Lenovo\Desktop\git\DataSet.zip\DataSet\WithoutMask")

In [4]:
with_mask_images = np.array(with_mask_images)
without_mask_images = np.array(without_mask_images)

In [5]:
with_mask_labels = np.ones(len(with_mask_images))
without_mask_labels = np.zeros(len(without_mask_images))

In [6]:
X_train = np.concatenate((with_mask_images[:8000], without_mask_images[:8000]), axis=0)
y_train = np.concatenate((with_mask_labels[:8000], without_mask_labels[:8000]), axis=0)
X_test = np.concatenate((with_mask_images[8000:], without_mask_images[8000:]), axis=0)
y_test = np.concatenate((with_mask_labels[8000:], without_mask_labels[8000:]), axis=0)

In [7]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
31/31 [==============================] - 14s 374ms/step - loss: 0.8546 - accuracy: 0.5694
Epoch 2/10
31/31 [==============================] - 11s 346ms/step - loss: 0.4126 - accuracy: 0.8375
Epoch 3/10
31/31 [==============================] - 10s 339ms/step - loss: 0.2503 - accuracy: 0.9079
Epoch 4/10
31/31 [==============================] - 11s 343ms/step - loss: 0.2280 - accuracy: 0.9193
Epoch 5/10
31/31 [==============================] - 11s 343ms/step - loss: 0.1720 - accuracy: 0.9369
Epoch 6/10
31/31 [==============================] - 11s 346ms/step - loss: 0.1319 - accuracy: 0.9524
Epoch 7/10
31/31 [==============================] - 11s 346ms/step - loss: 0.1248 - accuracy: 0.9596
Epoch 8/10
31/31 [==============================] - 11s 342ms/step - loss: 0.1471 - accuracy: 0.9534
Epoch 9/10
31/31 [==============================] - 11s 339ms/step - loss: 0.1419 - accuracy: 0.9565
Epoch 10/10
31/31 [==============================] - 11s 343ms/step - loss: 0.1258 - accura

In [11]:
model.save('mask_detection_model.h5')

In [13]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_roi = frame[y:y + h, x:x + w]
        face_roi = cv2.resize(face_roi, (128, 128))
        face_roi = np.expand_dims(face_roi, axis=0)
        face_roi = face_roi / 255.0

        prediction = model.predict(face_roi)
        if prediction > 0.5:
            label = 'With Mask'
        else:
            label = 'Without Mask'

        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 30ms/step
